## **Demo Notebook**

This quick notebook will show how to

* load config
* create the dataset
* make a model
* load pretrained weights

This notebook should be run directly from the `scripts/` directory.

### **Config**

Make sure to change the following paths
* `DATASET_DIR`
* `LABELS_DIR`

In [11]:
from hydra import core, initialize, compose
from omegaconf import OmegaConf


# CHANGE ME
DATASET_DIR = '/home/kimdj98/my_folder/datasets/nuscenes'
LABELS_DIR = '/home/kimdj98/my_folder/datasets/cvt_labels_nuscenes_v2'


core.global_hydra.GlobalHydra.instance().clear()        # required for Hydra in notebooks

initialize(config_path='../config')

# Add additional command line overrides
cfg = compose(
    config_name='config',
    overrides=[
        'experiment.save_dir=../logs/',                 # required for Hydra in notebooks
        '+experiment=cvt_nuscenes_vehicle',
        f'data.dataset_dir={DATASET_DIR}',
        f'data.labels_dir={LABELS_DIR}',
        'data.version=v1.0-trainval',
        'loader.batch_size=1',
    ]
)

# resolve config references
OmegaConf.resolve(cfg)

print(list(cfg.keys()))

['experiment', 'loader', 'optimizer', 'scheduler', 'trainer', 'model', 'data', 'visualization', 'loss', 'metrics']


### **Dataset Setup**

In [12]:
import torch
import numpy as np

from cross_view_transformer.common import setup_experiment, load_backbone


# Additional splits can be added to cross_view_transformer/data/splits/nuscenes/
SPLIT = 'val_qualitative_000'
SUBSAMPLE = 5


model, data, viz = setup_experiment(cfg)

dataset = data.get_split(SPLIT, loader=False)
dataset = torch.utils.data.ConcatDataset(dataset)
dataset = torch.utils.data.Subset(dataset, range(0, len(dataset), SUBSAMPLE))

loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

print(len(dataset))

Loaded pretrained weights for efficientnet-b4
49


In [13]:
viz.visualize(dataset[0])

<generator object BaseViz.visualize at 0x7f1888d6bd60>

### **Network Setup**

In [14]:
from pathlib import Path


# Download a pretrained model (13 Mb)
MODEL_URL = 'https://www.cs.utexas.edu/~bzhou/cvt/cvt_nuscenes_vehicles_50k.ckpt'
CHECKPOINT_PATH = '../logs/cvt_nuscenes_vehicles_50k.ckpt'


# download it one time!
# after you download you can comment this out

# !wget $MODEL_URL -O $CHECKPOINT_PATH


if Path(CHECKPOINT_PATH).exists():
    network = load_backbone(CHECKPOINT_PATH)
else:
    network = model.backbone

    print(f'{CHECKPOINT_PATH} not found. Using randomly initialized weights.')

3023.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


--2023-05-21 02:09:15--  https://www.cs.utexas.edu/~bzhou/cvt/cvt_nuscenes_vehicles_50k.ckpt
Resolving www.cs.utexas.edu (www.cs.utexas.edu)... 128.83.120.48
Connecting to www.cs.utexas.edu (www.cs.utexas.edu)|128.83.120.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13407137 (13M)
Saving to: ‘../logs/cvt_nuscenes_vehicles_50k.ckpt’

../logs/cvt_nuscene 100%[===================>]  12.79M  5.22MB/s    in 2.4s    

2023-05-21 02:09:18 (5.22 MB/s) - ‘../logs/cvt_nuscenes_vehicles_50k.ckpt’ saved [13407137/13407137]

Loaded pretrained weights for efficientnet-b4


### **Run the Model**

This cell will save the model predictions and render them as a gif.

Note: colors will be ugly since gif only supports 256 colors total

In [16]:
import torch
import time
import imageio
import ipywidgets as widgets


GIF_PATH = './predictions.gif'


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network.to(device)
network.eval()

images = list()

with torch.no_grad():
    for batch in loader:
        
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        pred = network(batch)

        # visualization = np.vstack(viz(batch=batch, pred=pred))
        visualization = np.vstack(viz(batch=batch))
        images.append(visualization)


# Save a gif
duration = [1 for _ in images[:-1]] + [5 for _ in images[-1:]]
imageio.mimsave(GIF_PATH, images, duration=duration)

html = f'''
<div align="center">
<img src="{GIF_PATH}?modified={time.time()}" width="80%">
</div>
'''

display(widgets.HTML(html))

HTML(value='\n<div align="center">\n<img src="./predictions.gif?modified=1684602621.921385" width="80%">\n</di…